In [31]:
import cv2
import numpy as np
import time

def detect_yolo_onnx(confidenc, image, input_width, input_height, class_list, model, model_path, model_details):
    models_returned = []
    INPUT_WIDTH = input_width
    INPUT_HEIGHT = input_height
    class_list = class_list
    net = cv2.dnn.readNet(model_path)
    row, col, _ = image.shape
    print(row, col)
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = image
    detection_start = time.perf_counter()
    blob = cv2.dnn.blobFromImage(result, 1/255.0, (INPUT_WIDTH, INPUT_HEIGHT), swapRB=True, crop=False)

    print(blob.shape)
    #write blob to file
    with open('./sample-images/blob.txt', 'w') as f:
        for item in blob[0][0][0]:
            f.write("%s\n" % item)
    

    net.setInput(blob)
    preds = net.forward()
    print(preds.shape)
    detection_finished = time.perf_counter()
    if model == "yolov8":
        preds = preds.transpose((0, 2, 1))
    print(preds.shape)

    class_ids = []
    confidences = []
    boxes = []
    rows = preds[0].shape[0]
    image_width, image_height, _ = result.shape
    x_factor = image_width / INPUT_WIDTH
    y_factor =  image_height / INPUT_HEIGHT
    list_of_conf = []
    print( f"Rows : {rows}")
    for r in range(rows):

        row = preds[0][r]
       # print length of row
        print(f"Row lenght:{r}")
        confidence = row[4]
        list_of_conf.append(confidence)
        if confidence >= confidenc:
            print(f"Confidence: {confidence}")
            if model == "yolov8":
                classes_scores = row[4:]
            else:
                classes_scores = row[5:]
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            if (classes_scores[class_id] > .25):
                confidences.append(confidence)
                class_ids.append(class_id)
                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.25, 0.45) 
    result_class_ids = []
    result_confidences = []
    result_boxes = []
    for i in indexes:
        result_confidences.append(confidences[i])
        result_class_ids.append(class_ids[i])
        result_boxes.append(boxes[i])
    
    colors = [(255, 255, 0), (0, 255, 0), (0, 255, 255), (255, 0, 0)]
    for (classid, confidence, box) in zip(result_class_ids, result_confidences, result_boxes):
         color = colors[int(classid) % len(colors)]
         cv2.rectangle(image, box, color, 5)
         cv2.rectangle(image, (box[0], box[1] - 20), (box[0] + box[2], box[1]), color, -1)
         cv2.putText(image, class_list[classid], (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,0))
    yolo_onnx_returned = []
    #yolo_onnx_returned.append(model_details)
    if len(result_boxes) == 0:
        yolo_onnx_returned.append(f"Detection attempt took {round(detection_finished - detection_start,2)} seconds. No tattos detected. Try setting confindence level to {(round(np.amax(list_of_conf) * 100) -1)} or lower.")
    else:
        yolo_onnx_returned.append(f"Model detected {len(result_boxes)} tattoos in {round(detection_finished - detection_start,2)} seconds with a confidence level of {confidenc * 100} percent.")
    yolo_onnx_returned.append(image)

    cv2.imwrite('./sample-images/python-output-2.jpg',image)
    models_returned.append(yolo_onnx_returned)
    return models_returned

class_list = ["car"]
img = cv2.imread("./sample-images/car-with-licenseplate.jpeg")
ouput_image = detect_yolo_onnx(0.25,img, 640, 640, class_list, 'yolov8', './models/lpr-8n.onnx', 'yolov8')
print(ouput_image[0][0])


4032 3024
(1, 3, 640, 640)
(1, 5, 8400)
(1, 8400, 5)
Rows : 8400
Row lenght:0
Row lenght:1
Row lenght:2
Row lenght:3
Row lenght:4
Row lenght:5
Row lenght:6
Row lenght:7
Row lenght:8
Row lenght:9
Row lenght:10
Row lenght:11
Row lenght:12
Row lenght:13
Row lenght:14
Row lenght:15
Row lenght:16
Row lenght:17
Row lenght:18
Row lenght:19
Row lenght:20
Row lenght:21
Row lenght:22
Row lenght:23
Row lenght:24
Row lenght:25
Row lenght:26
Row lenght:27
Row lenght:28
Row lenght:29
Row lenght:30
Row lenght:31
Row lenght:32
Row lenght:33
Row lenght:34
Row lenght:35
Row lenght:36
Row lenght:37
Row lenght:38
Row lenght:39
Row lenght:40
Row lenght:41
Row lenght:42
Row lenght:43
Row lenght:44
Row lenght:45
Row lenght:46
Row lenght:47
Row lenght:48
Row lenght:49
Row lenght:50
Row lenght:51
Row lenght:52
Row lenght:53
Row lenght:54
Row lenght:55
Row lenght:56
Row lenght:57
Row lenght:58
Row lenght:59
Row lenght:60
Row lenght:61
Row lenght:62
Row lenght:63
Row lenght:64
Row lenght:65
Row lenght:66
Row len